In [1]:
#import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

#불러온 모듈 webdriver에서 Chrom()객체를 생성


In [2]:
#링크 접속
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.melon.com/chart/index.htm")
# 창 크기 설정
# 요소를 찾고 클릭하는 동작에 대해 최대 3초 동안 대기
driver.implicitly_wait(3)

# 창 크기 가져오기 및 출력
size = driver.get_window_size()
width = size.get("width")
height = size.get("height")
print(f"{width}px {height}px")

# 창 크기 설정
driver.set_window_size(1400, 1346)




1050px 1000px


In [64]:
#차트 파인더
button = driver.find_element(By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span')
ActionChains(driver).click(button).perform()

#주간차트
button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/h4[1]')
ActionChains(driver).click(button).perform()
#연대
button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span')
ActionChains(driver).click(button).perform()

#연도
button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[1]')
ActionChains(driver).click(button).perform()

#월간

button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span')
ActionChains(driver).click(button).perform()
#주간
button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[1]/span')
ActionChains(driver).click(button).perform()
#장르
button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span')
ActionChains(driver).click(button).perform()

#버튼
button = driver.find_element(By.XPATH, '//*[@id="d_srch_form"]/div[2]/button/span')
ActionChains(driver).click(button).perform()


In [177]:
title =[]
album = []
singer = []
date = []
like = []
rank = []

In [188]:
print(len(title))
print(len(singer))
print(len(album))
print(len(date))
print(len(like))
print(len(rank))

400
400
400
400
400
100


In [182]:
# 각자 가져오는 방법, 이거부터 실행
driver.implicitly_wait(3)
html_source = driver.page_source

parse = BeautifulSoup(html_source, 'html.parser')

titles = parse.find_all("div", {"class": "ellipsis rank01"}) 
singers = parse.find_all("div", {"class": "ellipsis rank02"}) 
albums = parse.find_all("div",{"class": "ellipsis rank03"}) 
dates = driver.find_element(By.CLASS_NAME,'datelk').text

likes_elements = parse.find_all("span", {"class": "cnt"})  # 좋아요 수 추출
likes = [element.text.strip() for element in likes_elements]


rank = [i for i in range(1,101)]


for t in titles:
    title.append(t.find('a').text) 
for s in singers:
    singer.append(s.find('span', {"class": "checkEllipsis"}).text)

for a in albums:
    album.append(a.find('a').text)
    date.append(dates)
for l in likes:
    l = (re.sub(r'\D', '', l.replace("총건수", "").strip()))
    like.append(l)


for i in range(len(rank)):
    print('%3d위: %s [ %s ] - %s'%(i+1, title[i], album[i], singer[i]),date[i],like[i])



  1위: 비의 랩소디 [ 비의 랩소디 ] - 임재현 2024.01.22 ~ 2024.01.28 68649
  2위: To. X [ To. X - The 5th Mini Album ] - 태연 (TAEYEON) 2024.01.22 ~ 2024.01.28 143625
  3위: Love wins all [ The Winning ] - 아이유 2024.01.22 ~ 2024.01.28 160297
  4위: Perfect Night [ Perfect Night ] - LE SSERAFIM (르세라핌) 2024.01.22 ~ 2024.01.28 114097
  5위: Drama [ Drama - The 4th Mini Album ] - aespa 2024.01.22 ~ 2024.01.28 85678
  6위: 에피소드 [ 에피소드 ] - 이무진 2024.01.22 ~ 2024.01.28 68244
  7위: Love 119 [ Love 119 ] - RIIZE 2024.01.22 ~ 2024.01.28 95079
  8위: 그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection)) [ 그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection)) ] - 너드커넥션 (Nerd Connection) 2024.01.22 ~ 2024.01.28 97933
  9위: 헤어지자 말해요 [ 1집 Alone ] - 박재정 2024.01.22 ~ 2024.01.28 169614
 10위: 인사 [ 인사 ] - 범진 2024.01.22 ~ 2024.01.28 107604
 11위: Baddie [ I'VE MINE ] - IVE (아이브) 2024.01.22 ~ 2024.01.28 68602
 12위: Seven (feat. Latto) - Clean Ver. [ Seven (feat. Latto) - Clean Ver. ] - 정국 2024.01.22 ~ 2024.01.28 208038
 13위: I AM [ I've IVE ] - IVE (아이브)

In [187]:
data = {
        'rank' : rank * (len(title) // 100),
        'title': title,
        'singer': singer,
        'album': album,
        'dates': date,
        'like' : like, 
        
    }

chart_df = pd.DataFrame(data)
chart_df.to_csv('melon_chart.csv', index=False, encoding='utf-8-sig')